
# Fine-tuning T5 Summarization

---

In [1]:
!nvidia-smi

Sun Apr 10 17:06:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Setup

---

In [2]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [3]:
! pip install transformers
! pip install datasets
! pip install sentencepiece
! pip install rouge_score
! pip install wandb

     |████████████████████████████████| 4.0 MB 7.8 MB/s 
     |████████████████████████████████| 6.5 MB 42.2 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
     |████████████████████████████████| 895 kB 63.7 MB/s 
     |████████████████████████████████| 596 kB 60.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 9.6 MB/s 
     |████████████████████████████████| 136 kB 67.0 MB/s 
     |████████████████████████████████| 212 kB 65.8 MB/s 
     |████████████████████████████████| 1.1 MB 55.8 MB/s 
     |████████████████████████████████| 127 kB 74.9 MB/s 
     |████████████████████████████████| 144 kB 72.0 MB/s 
     |████████████████████████████████| 94 kB 4.3 MB/s 
     |████████████████████████████████| 271 kB 72.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [4]:
import torch
import numpy as np
import datasets

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

from tabulate import tabulate
import nltk
from datetime import datetime

In [5]:
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
    import wandb

    wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Model and tokenizer

---

Hiperparámetros: 

[HF Bart configuration](https://huggingface.co/transformers/_modules/transformers/configuration_bart.html)

[Fairseq Bart](https://github.com/pytorch/fairseq/tree/master/examples/bart)

In [6]:
import tensorflow as tf
print(tf.test.is_built_with_cuda())

True


In [7]:
#Llamado del modelo
model_name = "t5-small"
#Definición de modelo y tokenizador
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Se fijan los parámetros del modelo
model.config.activation_dropout = 0.1
model.config.classif_dropout = 0.0
model.config.max_encoder_position_embeddings = 1024
model.config.max_decoder_position_embeddings = 16384
model.config.min_length = 350
model.config.max_length = 500
print(model.config)

# tokenización
encoder_max_length = 256 
decoder_max_length = 64

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

T5Config {
  "_name_or_path": "t5-small",
  "activation_dropout": 0.1,
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "classif_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "max_decoder_position_embeddings": 16384,
  "max_encoder_position_embeddings": 1024,
  "max_length": 500,
  "min_length": 350,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
  

## Data

---

In [8]:
#Lista de datasets compatibles con la biblioteca
# datasets.list_datasets()

### Descarga y Preparación de los Datos

### Cargado de Dataset

In [9]:
train_data_txt = datasets.load_dataset("ccdv/govreport-summarization", split="train")
validation_data_txt = datasets.load_dataset("ccdv/govreport-summarization", split="test")

No config specified, defaulting to: gov_report_summarization_dataset/document


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset gov_report_summarization_dataset downloaded and prepared to /root/.cache/huggingface/datasets/ccdv___gov_report_summarization_dataset/document/1.0.0/32c7f04ec577e9dc99b889eff9d64a321a3a00b7332e8733eb32d39bebbdc34f. Subsequent calls will reuse this data.


No config specified, defaulting to: gov_report_summarization_dataset/document
Reusing dataset gov_report_summarization_dataset (/root/.cache/huggingface/datasets/ccdv___gov_report_summarization_dataset/document/1.0.0/32c7f04ec577e9dc99b889eff9d64a321a3a00b7332e8733eb32d39bebbdc34f)


**Preprocess and tokenize**

In [10]:
def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source, target = batch["report"], batch["summary"]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch


train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)

  0%|          | 0/18 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Training

---

### Metrics

In [11]:
# Borrowed from https://github.com/huggingface/transformers/blob/master/examples/seq2seq/run_summarization.py

nltk.download("punkt", quiet=True)

metric = datasets.load_metric("rouge")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

### Training arguments

In [12]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=25,  # Deseado: 10
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=32,  # numeros al poder de 2, 64 o 32 son buenos valores, intentar ambos
    per_device_eval_batch_size=32,
    learning_rate=3e-04, #3e-05 puede ser tambien
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True, #Para métricas ROUGE
    logging_dir="logs",
    logging_steps=500,
    save_total_limit=3, #guardar ultimos 3 checkpoints
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

### Train

Wandb integration

In [13]:
wandb

<module 'wandb' from '/usr/local/lib/python3.7/dist-packages/wandb/__init__.py'>

In [14]:
if WANDB_INTEGRATION:
    wandb_run = wandb.init(
        project="T5_GOVREP_FT",
        config={
            "per_device_train_batch_size": training_args.per_device_train_batch_size,
            "learning_rate": training_args.learning_rate,
            "dataset": "ccdv/govreport-summarization",
        },
    )

    now = datetime.now()
    current_time = now.strftime("%H%M%S")
    wandb_run.name = "run_" + current_time

wandb: Currently logged in as: zeroranger (use `wandb login --relogin` to force relogin)


Evaluate before fine-tuning

In [15]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 973
  Batch size = 32


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


{'eval_gen_len': 396.259,
 'eval_loss': 5.236630439758301,
 'eval_rouge1': 27.2619,
 'eval_rouge2': 8.2413,
 'eval_rougeL': 18.6114,
 'eval_rougeLsum': 23.1329,
 'eval_runtime': 169.4094,
 'eval_samples_per_second': 5.743,
 'eval_steps_per_second': 0.183}

Train the model

In [16]:
torch.cuda.is_available()
import tensorflow as tf

In [17]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [18]:
#%%wandb
# uncomment to display Wandb charts

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 17517
  Num Epochs = 25
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 13700


Step,Training Loss
500,4.285200
1000,3.895400
1500,3.768300
2000,3.692700
2500,3.628000
3000,3.572800
3500,3.519900
4000,3.485300
4500,3.448700
5000,3.416200


Saving model checkpoint to results/checkpoint-500
Configuration saved in results/checkpoint-500/config.json
Model weights saved in results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-500/tokenizer_config.json
Special tokens file saved in results/checkpoint-500/special_tokens_map.json
Copy vocab file to results/checkpoint-500/spiece.model
Saving model checkpoint to results/checkpoint-1000
Configuration saved in results/checkpoint-1000/config.json
Model weights saved in results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in results/checkpoint-1000/special_tokens_map.json
Copy vocab file to results/checkpoint-1000/spiece.model
Saving model checkpoint to results/checkpoint-1500
Configuration saved in results/checkpoint-1500/config.json
Model weights saved in results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1500/tokeni

TrainOutput(global_step=13700, training_loss=3.3929365751169023, metrics={'train_runtime': 8121.2408, 'train_samples_per_second': 53.923, 'train_steps_per_second': 1.687, 'total_flos': 2.96347792048128e+16, 'train_loss': 3.3929365751169023, 'epoch': 25.0})

Evaluate after fine-tuning

In [19]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 973
  Batch size = 32


{'epoch': 25.0,
 'eval_gen_len': 355.8191,
 'eval_loss': 3.609917402267456,
 'eval_rouge1': 11.0406,
 'eval_rouge2': 4.7406,
 'eval_rougeL': 8.0941,
 'eval_rougeLsum': 9.7144,
 'eval_runtime': 175.2481,
 'eval_samples_per_second': 5.552,
 'eval_steps_per_second': 0.177}

In [20]:
if WANDB_INTEGRATION:
    wandb_run.finish()

eval/gen_len,█▁
eval/loss,█▁
eval/rouge1,█▁
eval/rouge2,█▁
eval/rougeL,█▁
eval/rougeLsum,█▁
eval/runtime,▁█
eval/samples_per_second,█▁
eval/steps_per_second,█▁
train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇████
train/global_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇████


## Evaluation

---

**Generate summaries from the fine-tuned model and compare them with those generated from the original, pre-trained one.**

In [21]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["report"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)

test_samples = validation_data_txt.select(range(16))

summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1]

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_lengt

In [22]:
print(
    tabulate(
        zip(
            range(len(summaries_after_tuning)),
            summaries_after_tuning,
            summaries_before_tuning,
        ),
        headers=["Id", "Summary after", "Summary before"],
    )
)
print("\nTarget summaries:\n")
print(
    tabulate(list(enumerate(test_samples["summary"])), headers=["Id", "Target summary"])
)
# print("\nSource documents:\n")
# print(tabulate(list(enumerate(test_samples["text"])), headers=["Id", "Document"]))

  Id  Summary after                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
from google.colab import files

torch.save(model.state_dict(), 'checkpointT5GR.pth')

# download checkpoint file
files.download('checkpointT5GR.pth')